<a href="https://colab.research.google.com/github/ymiftah/operations_research/blob/master/cpu_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qq pyomo
!apt-get install -qq coinor-cbc

     |████████████████████████████████| 2.3MB 2.8MB/s 
     |████████████████████████████████| 256kB 46.2MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 163kB 41.2MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 132681 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coi

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

# CPU Scheduling

## Problem Statement
10 tasks must be run on 3 CPUs at 1.33, 2 and 2.66 GHz (each processor can run only one task at a time). The number of elementary operations of the tasks (expressed in billions of instructions (BI)) is as follows:


|    Process  | 1|2|3|4|5|6|7  |
|---------|---|---|---|---|---|---|---|
|BI   | 1.1| 2.1| 3| 1|0.7|5|3|

**Schedule tasks to processors so that the completion time of the last task is minimized**

In [0]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

cpus =   ['1.33 GHz', '2 GHz', '2.66 GHz']
freq = dict(zip(cpus, [1.33, 2, 2.66]))

process = [str(i) for i in range(1,8)]
BI = dict(zip(process, [1.1, 2.1, 3, 1, 0.7, 5, 3]))


# Sets
model.cpus = Set(initialize=cpus)
model.process = Set(initialize=process)

# declare decision variables
model.to_cpu = Var(model.cpus, model.process, domain=Binary)
model.last_time = Var(domain=NonNegativeReals)

# declare objective
model.obj = Objective(expr = model.last_time,
                       sense=minimize)

#declare constraints
def cstr_assign(m, p):
    # Each task is assigned to exactly one cpu
    return (sum(m.to_cpu[c, p] for c in m.cpus) == 1)

def cstr_time(m, c):
    # the last_time is the upper bound of the time taken by
    # each cpu to complete the tasks assigned to it
    return (sum(BI[p] * m.to_cpu[c, p] / freq[c]
               for p in m.process) <= m.last_time)

model.assign = Constraint(model.process, rule=cstr_assign)
model.min_time = Constraint(model.cpus, rule=cstr_time)

In [11]:
# Solve the problem
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model)

print('Completion time = ', model.obj(), 'seconds')

Completion time =  2.7819549 seconds


## Results

Let us visualize in a table the task assignment

In [17]:
assignment = model.to_cpu.extract_values()

df = pd.DataFrame(
    [[assignment[i,j] for i in cpus] for j in process],
    columns=cpus,
    index=process,
    dtype='int'
)
print(df)

   1.33 GHz  2 GHz  2.66 GHz
1         0      0         1
2         0      0         1
3         1      0         0
4         0      0         1
5         1      0         0
6         0      1         0
7         0      0         1


The completion time for all tasks is 2.78 seconds

In [19]:
print('Completion time = ', model.obj(), 'seconds')

Completion time =  2.7819549 seconds
